In [1]:
# 將財產分配 load 程式
# 此程式用 geocoder.osm(place).latlng 將「中文地名」換算成 gps， 必須上外網且 OA 不能用。

import pandas as pd
import numpy as np
import time
from datetime import date



def filter_address(value):
    if value[:3]=="嘉義市":
        return value[0:5]
    else:
        return value[0:6]
def filter_color(value):
    if value[5:7]=="01" or value[5:7]=="07" :
        value = 'purple'
    elif value[5:7]=="02" or value[5:7]=="08":
        value = 'blue'
    elif value[5:7]=="03" or value[5:7]=="09":
        value = 'green'
    elif value[5:7]=="04" or value[5:7]=="10":
        value = 'orange'
    elif value[5:7]=="05" or value[5:7]=="11":
        value = 'pink' 
    else:
        value = 'beige'
    return value
    
def find_earlier_data(now1):
    path= './data_complain'
    date_list = os.listdir(path)
    
    i=0    
    for i in range(60):
        date_name = 'complain({}).xlsx'.format(now1)
        if date_name in date_list:
            break 
        else:
            now1 = now1 - datetime.timedelta(days=1)
    return(date_name)  

def find_earlier_eNodebList():
    list1=['','','']
    path= './data_eNodebList'
    date_list = os.listdir(path)
    n=0
    now1= date.today() 
    for i in range(60):   
        date_name_4G = 'eNodebList_4G({}).xlsx'.format(now1)
        date_name_5G = 'eNodebList_5G({}).xlsx'.format(now1)
        date_name_3G = 'eNodebList_3G({}).xlsx'.format(now1)
        if date_name_4G in date_list:
            if list1[0] =='':
                list1[0] = date_name_4G
                n=n+1
        if date_name_5G in date_list:
            if list1[1] =='':
                list1[1] = date_name_5G
                n=n+1
        if date_name_3G in date_list:
            if list1[2] =='':
                list1[2] = date_name_3G
                n=n+1
        if n==3:
            break
        else:
            now1 = now1 - datetime.timedelta(days=1)
    return(list1)

today = date.today() 
#-------------------1. 讀取 3G 站台資料 & 分析 -----------------------#  
file_3G = find_earlier_eNodebList()[2] 
file1_path = "./data_eNodebList/{}".format(file_3G)
file_3G_df = pd.read_excel(file1_path)

file_3G_df = file_3G_df[['基地台編號(baseid)','細胞名稱(Cell_Name)','細胞經度(Cell_Lon)','細胞緯度(Cell_Lat)',\
                         '細胞縣市(Cell_County)','細胞鄉鎮(Cell_Town)']]
file_3G_df.drop_duplicates(subset=["基地台編號(baseid)","細胞名稱(Cell_Name)"],keep='first',inplace=True)
file_3G_df.reset_index(inplace=True, drop=True)
file_3G_df['基地台_fullname'] = file_3G_df['基地台編號(baseid)']+'('+file_3G_df['細胞名稱(Cell_Name)']+')'
file_3G_df['新地址'] = file_3G_df['細胞縣市(Cell_County)']+file_3G_df['細胞鄉鎮(Cell_Town)']
file_3G_df['新地址'] = file_3G_df['新地址'].map(filter_address)
file_3G_df = file_3G_df.drop(['基地台編號(baseid)','細胞名稱(Cell_Name)','細胞縣市(Cell_County)','細胞鄉鎮(Cell_Town)'],axis=1)
# file_3G_df column name 剩下 ['基地台_fullname','新地址','細胞經度(Cell_Lon)','細胞緯度(Cell_Lat)']
file_3G_df = file_3G_df[['基地台_fullname','新地址','細胞經度(Cell_Lon)','細胞緯度(Cell_Lat)']]


#-------------------2. 讀取 4G 站台資料 & 分析 -----------------------#  
file_4G = find_earlier_eNodebList()[0] 
file1_path = "./data_eNodebList/{}".format(file_4G)
file_4G_df = pd.read_excel(file1_path)

file_4G_df = file_4G_df[['基地台編號(baseid)','細胞名稱(Cell_Name)','細胞東經(Cell_Lon)','細胞北緯(Cell_Lat)',\
                         '細胞縣市(Cell_County)','細胞鄉鎮(Cell_Town)']]
file_4G_df.drop_duplicates(subset=["基地台編號(baseid)","細胞名稱(Cell_Name)"],keep='first',inplace=True)
file_4G_df.reset_index(inplace=True, drop=True)
file_4G_df['基地台_fullname'] = file_4G_df['基地台編號(baseid)']+'('+file_4G_df['細胞名稱(Cell_Name)']+')'
file_4G_df['新地址'] = file_4G_df['細胞縣市(Cell_County)']+file_4G_df['細胞鄉鎮(Cell_Town)']
file_4G_df['新地址'] = file_4G_df['新地址'].map(filter_address)
file_4G_df = file_4G_df.drop(['基地台編號(baseid)','細胞名稱(Cell_Name)','細胞縣市(Cell_County)','細胞鄉鎮(Cell_Town)'],axis=1)
# file_4G_df column name 剩下 ['基地台_fullname','新地址','細胞東經(Cell_Lon)','細胞北緯(Cell_Lat)']
file_4G_df = file_4G_df[['基地台_fullname','新地址','細胞東經(Cell_Lon)','細胞北緯(Cell_Lat)']]

#-----------------3. 讀取 5G 站台資料 & 分析-------------------------#  
file_5G = find_earlier_eNodebList()[1]
file1_path = "./data_eNodebList/{}".format(file_5G)
file_5G_df = pd.read_excel(file1_path)

file_5G_df = file_5G_df[['基地台編號(BaseID)','細胞名稱(Cell_Name)','細胞東經(Cell_Lon)','細胞北緯(Cell_Lat)',\
                         '細胞縣市(Cell_County)','細胞鄉鎮市區(Cell_Town)']]
file_5G_df.drop_duplicates(subset=["基地台編號(BaseID)","細胞名稱(Cell_Name)"],keep='first',inplace=True)
file_5G_df.reset_index(inplace=True, drop=True)
file_5G_df['基地台_fullname'] = file_5G_df['基地台編號(BaseID)']+'('+file_5G_df['細胞名稱(Cell_Name)']+')'
file_5G_df['新地址'] = file_5G_df['細胞縣市(Cell_County)']+file_5G_df['細胞鄉鎮市區(Cell_Town)']
file_5G_df['新地址'] = file_5G_df['新地址'].map(filter_address)
file_5G_df = file_5G_df.drop(['基地台編號(BaseID)','細胞名稱(Cell_Name)','細胞縣市(Cell_County)','細胞鄉鎮市區(Cell_Town)'],axis=1)
# file_5G_df column name 剩下 ['基地台_fullname','新地址','細胞東經(Cell_Lon)','細胞北緯(Cell_Lat)']
file_5G_df = file_5G_df[['基地台_fullname','新地址','細胞東經(Cell_Lon)','細胞北緯(Cell_Lat)']]

#-----------------3. 讀取 complain 檔案名稱 & 分析---------------#
used_day = find_earlier_data(today) # 使用 def 
file1_path = "./data_complain/{}".format(used_day)
xls_df = pd.read_excel(file1_path)

xls_df = xls_df[['編號','新地址','經度','緯度','地址資料不全']]
xls_df = xls_df[xls_df['地址資料不全']=='否']
xls_df['新地址'] =xls_df['新地址'].map(filter_address)
xls_df = xls_df.drop('地址資料不全',axis=1)
xls_df['顏色'] = xls_df['編號'].map(filter_color)

#------------------設定 icon-------------------------#
# myIcon = folium.CustomIcon('header-icon.png',icon_size = (30, 30),icon_anchor = (15, 30))
                          
#---------製作地圖-----------------------------------#
        
section_name = list(xls_df['新地址'].drop_duplicates())   
for i in section_name:
    gps = geocoder.osm(i).latlng
    m = folium.Map(gps, zoom_start=16,control_scale=True)
    folium.Marker(gps, popup=i, icon=folium.Icon(color="red")).add_to(m)
    complain_df = xls_df[xls_df['新地址']== i]
    complain_df.reset_index(inplace=True, drop=True)
    for_4G_df = file_4G_df[file_4G_df['新地址']== i]
    for_4G_df.reset_index(inplace=True, drop=True)
    for_5G_df = file_5G_df[file_5G_df['新地址']== i]
    for_5G_df.reset_index(inplace=True, drop=True)
    for_3G_df = file_3G_df[file_3G_df['新地址']== i]
    for_3G_df.reset_index(inplace=True, drop=True)
    
    for n in range(len(complain_df.index)):
        lng = complain_df['經度'].loc[n]
        lat = complain_df['緯度'].loc[n]
        custom_order = complain_df['編號'].loc[n]
        a_color = complain_df['顏色'].loc[n]
        folium.Marker((lat,lng),popup=custom_order,icon=folium.Icon(color=a_color,angle=90)).add_to(m)
    
    for n in range(len(for_5G_df.index)):
        lng = for_5G_df['細胞東經(Cell_Lon)'].loc[n]
        lat = for_5G_df['細胞北緯(Cell_Lat)'].loc[n]
        bts_order = for_5G_df['基地台_fullname'].loc[n]
        folium.Circle(radius=60,location=[lat,lng],popup=bts_order,color='darkblue',fill=True,fill_opacity=0.1).add_to(m)    
    
    for n in range(len(for_4G_df.index)):
        lng = for_4G_df['細胞東經(Cell_Lon)'].loc[n]
        lat = for_4G_df['細胞北緯(Cell_Lat)'].loc[n]
        bts_order = for_4G_df['基地台_fullname'].loc[n]
        folium.Circle(radius=40,location=[lat,lng],popup=bts_order,color='darkred',fill=True,fill_opacity=0.1).add_to(m)
             
    for n in range(len(for_3G_df.index)):
        lng = for_3G_df['細胞經度(Cell_Lon)'].loc[n]
        lat = for_3G_df['細胞緯度(Cell_Lat)'].loc[n]
        bts_order = for_3G_df['基地台_fullname'].loc[n]
        folium.Circle(radius=20,location=[lat,lng],popup=bts_order,color='darkgreen',fill=True,fill_opacity=0.1).add_to(m)        

    m.save('./result(complain)/'+f"{i}({today})"+".html")


# 